### 1. Solicitudes de tarjetas de crédito
Los bancos comerciales reciben muchas solicitudes de tarjetas de crédito. Muchos de ellos son rechazados por muchas razones, como saldos elevados de préstamos, bajos niveles de ingresos o demasiadas consultas sobre el informe crediticio de una persona, por ejemplo. El análisis manual de estas aplicaciones es mundano, propenso a errores y requiere mucho tiempo (¡y el tiempo es dinero!). Afortunadamente, esta tarea se puede automatizar con el poder del aprendizaje automático y prácticamente todos los bancos comerciales lo hacen hoy en día. En este cuaderno, construiremos un predictor automático de aprobación de tarjetas de crédito utilizando técnicas de aprendizaje automático, tal como lo hacen los bancos reales.

<img src=https://assets.datacamp.com/production/project_558/img/credit_card.jpg>

Usaremos el conjunto de datos de aprobación de tarjeta de crédito del repositorio de aprendizaje automático de UCI. La estructura de este cuaderno es la siguiente:

- Primero, comenzaremos cargando y viendo el conjunto de datos.

- Veremos que el conjunto de datos tiene una combinación de características numéricas y no numéricas, que contiene valores de diferentes rangos, además de que contiene varias entradas faltantes.

- Tendremos que preprocesar el conjunto de datos para garantizar que el modelo de aprendizaje automático que elijamos pueda hacer buenas predicciones.

- Una vez que nuestros datos estén en buen estado, haremos un análisis exploratorio de datos para construir nuestras intuiciones.
Finalmente, construiremos un modelo de aprendizaje automático que puede predecir si se aceptará la solicitud de una tarjeta de crédito de un individuo.

Primero, cargar y ver el conjunto de datos. Encontramos que dado que estos datos son confidenciales, el contribuyente del conjunto de datos ha anonimizado los nombres de las características.


In [2]:
# Import pandas
# ... YOUR CODE FOR TASK 1 ...
import pandas as pd
import numpy as np

# Load dataset
cc_apps = pd.read_csv("C:/Users/jorge/OneDrive/proyecto python Data Camp\Proyecto_tarjetas_credito/cc_approvals.data", header =None)

In [3]:
# Inspect data
# ... YOUR CODE FOR TASK 1 ...
cc_apps.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [4]:
# Replace the '?'s with NaN
cc_apps = cc_apps.replace('?', np.nan)

# Inspect the missing values again
cc_apps.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-
689,b,35.00,3.375,u,g,c,h,8.29,f,f,0,t,g,00000,0,-


### 2. Inspección de las aplicaciones
El resultado puede parecer un poco confuso a primera vista, pero intentemos descubrir las características más importantes de una solicitud de tarjeta de crédito. Las funciones de este conjunto de datos se han anonimizado para proteger la privacidad, pero este blog nos brinda una descripción general bastante buena de las funciones probables. Las características probables en una solicitud típica de tarjeta de crédito son Género, Edad, Deuda, Casado, Cliente bancario, Nivel de educación, Etnicidad, Años de empleo, Incumplimiento anterior, Empleado, Puntaje crediticio, Licencia de conducir, Ciudadano, Código postal, Ingresos y, finalmente, estado de aprobación. Esto nos brinda un buen punto de partida y podemos mapear estas características con respecto a las columnas en la salida.

Como podemos ver desde nuestro primer vistazo a los datos, el conjunto de datos tiene una mezcla de características numéricas y no numéricas. Esto se puede solucionar con un preprocesamiento, pero antes de hacerlo, aprendamos un poco más sobre el conjunto de datos para ver si hay otros problemas del conjunto de datos que deben solucionarse.

In [5]:
# Print summary statistics
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print("\n")

# Inspect missing values in the dataset
cc_apps.tail(17)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    object 
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 no

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


### 3. Manejo de los valores faltantes (parte i)
Hemos descubierto algunos problemas que afectarán el rendimiento de nuestros modelos de aprendizaje automático si no se modifican:

- Nuestro conjunto de datos contiene datos numéricos y no numéricos (específicamente datos que son de tipo float64, int64 y object). 

- Específicamente, las características 2, 7, 10 y 14 contienen valores numéricos (de tipo float64, float64, int64 e int64 respectivamente) y todas las demás características contienen valores no numéricos.

- El conjunto de datos también contiene valores de varios rangos. Algunas características tienen un rango de valores de 0 a 28, algunas tienen un rango de 2 a 67 y otras tienen un rango de 1017 a 100000. Además de esto, podemos obtener información estadística útil (como media, máx. y mín.) sobre las características que tienen valores numéricos.

- Finalmente, el conjunto de datos tiene valores faltantes, de los cuales nos ocuparemos en esta tarea. Los valores que faltan en el conjunto de datos están etiquetados con '?', que se puede ver en la salida de la última celda.

Ahora, reemplacemos temporalmente estos signos de interrogación de valor faltante con NaN.

In [6]:
# Import numpy
import numpy as np

# Inspect missing values in the dataset
print(cc_apps.isnull().values.sum())

# Replace the '?'s with NaN
cc_apps = cc_apps.replace('?', np.nan)

# Inspect the missing values again
cc_apps.tail(17)

67


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


### 4. Manejo de los valores faltantes (parte ii)
Reemplazamos todos los signos de interrogación con NaN. Esto nos va a ayudar en el próximo tratamiento de valores perdidos que vamos a realizar.

Una pregunta importante que surge aquí es ¿por qué le damos tanta importancia a los valores faltantes? ¿No pueden simplemente ser ignorados? Ignorar los valores faltantes puede afectar en gran medida el rendimiento de un modelo de aprendizaje automático. Al ignorar los valores faltantes, nuestro modelo de aprendizaje automático puede perder información sobre el conjunto de datos que puede ser útil para su entrenamiento. Entonces, hay muchos modelos que no pueden manejar los valores faltantes implícitamente, como LDA.

Entonces, para evitar este problema, vamos a imputar los valores faltantes con una estrategia llamada imputación media.

In [7]:
# Impute the missing values with mean imputation
cc_apps.fillna(cc_apps.mean(), inplace=True)

# Count the number of NaNs in the dataset to verify
print(cc_apps.isnull().values.sum())

67


### 5. Manejo de los valores faltantes (parte iii)
Nos hemos ocupado con éxito de los valores faltantes presentes en las columnas numéricas. Todavía faltan algunos valores por imputar para las columnas 0, 1, 3, 4, 5, 6 y 13. Todas estas columnas contienen datos no numéricos y es por eso que la estrategia de imputación media no funcionaría aquí. Esto necesita un tratamiento diferente.

Vamos a imputar estos valores perdidos con los valores más frecuentes presentes en las respectivas columnas. Esta es una buena práctica cuando se trata de imputar valores faltantes para datos categóricos en general.

In [8]:
# Iterar sobre cada columna de cc_apps
for col in cc_apps.columns:
    # Comprobar si la columna es de tipo objeto
    if cc_apps[col].dtypes == 'object':
        # Imputar con el valor más frecuente
        cc_apps = cc_apps.fillna(cc_apps[col].value_counts().index[0])

# Cuentae el número de NaN en el conjunto de datos e imprime el recuento para verificar
print(cc_apps.isnull().values.sum())

0


### 6. Preprocesamiento de los datos (parte i)
Los valores que faltan ahora se han manejan con éxito.

Todavía se necesita un preprocesamiento de datos menor pero esencial antes de proceder a construir nuestro modelo de aprendizaje automático. Vamos a dividir estos pasos de preprocesamiento restantes en tres tareas principales:

- Convierta los datos no numéricos en numéricos.
- Dividir los datos en conjuntos de entrenamiento y prueba.
- Escalar los valores de las características a un rango uniforme.

Primero, convertiremos todos los valores no numéricos en valores numéricos. Hacemos esto porque no solo da como resultado un cálculo más rápido, sino que también muchos modelos de aprendizaje automático (como XGBoost) (y especialmente los desarrollados con scikit-learn) requieren que los datos estén en un formato estrictamente numérico. Haremos esto usando una técnica llamada codificación de etiquetas.

In [9]:
# Importar LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Instanciar LabelEncoder
le = LabelEncoder()

# Iterar sobre todos los valores de cada columna y extraer sus dtypes
for col in cc_apps.columns:
    # Comparar si el dtype es object
    if cc_apps[col].dtypes=='object':
    # Use LabelEncoder para hacer la transformación numérica
        cc_apps[col]=le.fit_transform(cc_apps[col])
        
# Como podemos ver, todas las funciones ahora son de tipo numérico.
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    int32  
 1   1       690 non-null    int32  
 2   2       690 non-null    float64
 3   3       690 non-null    int32  
 4   4       690 non-null    int32  
 5   5       690 non-null    int32  
 6   6       690 non-null    int32  
 7   7       690 non-null    float64
 8   8       690 non-null    int32  
 9   9       690 non-null    int32  
 10  10      690 non-null    int64  
 11  11      690 non-null    int32  
 12  12      690 non-null    int32  
 13  13      690 non-null    int32  
 14  14      690 non-null    int64  
 15  15      690 non-null    int32  
dtypes: float64(2), int32(12), int64(2)
memory usage: 54.0 KB


### 7. Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
Ahora, dividiremos nuestros datos en un conjunto de entrenamiento y un conjunto de prueba para preparar nuestros datos para dos fases diferentes del modelado de aprendizaje automático: entrenamiento y prueba. Idealmente, no se debe usar información de los datos de prueba para preprocesar los datos de entrenamiento o para dirigir el proceso de entrenamiento de un modelo de aprendizaje automático. Por lo tanto, primero dividimos los datos y luego los preprocesamos.

Además, funciones como DriversLicense y ZipCode no son tan importantes como las otras funciones en el conjunto de datos para predecir aprobaciones de tarjetas de crédito. Para tener una mejor idea, podemos medir su correlación estadística con las etiquetas del conjunto de datos. Pero esto está fuera del alcance de este proyecto. Deberíamos eliminarlos para diseñar nuestro modelo de aprendizaje automático con el mejor conjunto de funciones. En la literatura de ciencia de datos, esto a menudo se denomina selección de características.

In [10]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
cc_apps = cc_apps.drop([cc_apps.columns[11], cc_apps.columns[13]], axis=1)
cc_apps = cc_apps.values

# Segregate features and labels into separate variables
X,y = cc_apps[:,0:13] , cc_apps[:,13]

# Split into train and test sets
cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)